In [6]:
def writeTiff(im_data,im_width,im_height,im_bands,im_geotrans,im_proj,path):
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32

    if len(im_data.shape) == 3:
        im_bands, im_height, im_width = im_data.shape
    elif len(im_data.shape) == 2:
        im_data = np.array([im_data])
    else:
        im_bands, (im_height, im_width) = 1,im_data.shape
        #创建文件
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(path, im_width, im_height, im_bands, datatype)
    if(dataset!= None):
        dataset.SetGeoTransform(im_geotrans) #写入仿射变换参数
        dataset.SetProjection(im_proj) #写入投影
    for i in range(im_bands):
        dataset.GetRasterBand(i+1).WriteArray(im_data[i])
    del dataset

In [7]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from osgeo import gdal

path = r'I:\haihe_clim\pr_haihe_historical_1970-2000.nc'

data = Dataset(path, 'r')

In [38]:
data['lon'][:], data['lat'][:]

(masked_array(data=[112.5, 113. , 113.5, 114. , 114.5, 115. , 115.5, 116. ,
                    116.5, 117. , 117.5, 118. , 118.5, 119. , 119.5],
              mask=False,
        fill_value=1e+20),
 masked_array(data=[34.5, 35. , 35.5, 36. , 36.5, 37. , 37.5, 38. , 38.5,
                    39. , 39.5, 40. , 40.5, 41. , 41.5, 42. , 42.5],
              mask=False,
        fill_value=1e+20))

In [31]:
data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    dimensions(sizes): lon(15), lat(17), time(11323), ens(50)
    variables(dimensions): float64 lon(lon), float64 lat(lat), float64 time(time), int32 ens(ens), float32 pr(ens,time,lat,lon), float32 q50(time,lat,lon), float32 q90(time,lat,lon), float32 q95(time,lat,lon)
    groups: 

In [69]:
a = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]*31
mon_day = np.array(a)
mon_day = np.insert(mon_day, 0, 0)

file_name = []
for i  in pd.date_range(start='19700101',end='20001231',freq='1M'):
    file_name.append(str(i)[:7].replace('-', '_'))

In [75]:
ds = gdal.Open(r'G:\china_geodata\China_soil_1km\ChinaSoil_1km.tif')
prj = ds.GetProjection()
#tr = ds.GetGeoTransform()
tr = (112.5, 0.5, 0.0, 42.5, 0.0, -0.5)
ind_0 = 0
ind_1 = 0
for i in range(372):
    ind_0 += mon_day[i]
    ind_1 += mon_day[i+1]
    a = np.mean(data['pr'][:,ind_0:ind_1,:,:], axis=0)
    b = np.sum(a, axis=0)
    writeTiff(b,15, 17, 1,tr, prj,r'I:\haihe_clim\pr_%s.tif'%file_name[i])